# 一、自然语言推断及数据集

在 15.1节中，我们讨论了情感分析问题。这个任务的目的是将单个文本序列分类到预定义的类别中，例如一组情感极性中。然而，当需要决定一个句子是否可以从另一个句子推断出来，或者需要通过识别语义等价的句子来消除句子间冗余时，知道如何对一个文本序列进行分类是不够的。相反，我们需要能够对成对的文本序列进行推断。

## 1.1 自然语言推断

自然语言推断（natural language inference）主要研究 假设（hypothesis）是否可以从前提（premise）中推断出来， 其中两者都是文本序列。 换言之，自然语言推断决定了一对文本序列之间的逻辑关系。这类关系通常分为三种类型：
* 蕴涵（entailment）：假设可以从前提中推断出来。
* 矛盾（contradiction）：假设的否定可以从前提中推断出来。
* 中性（neutral）：所有其他情况。

自然语言推断也被称为识别文本蕴涵任务。 例如，下面的一个文本对将被贴上“蕴涵”的标签，因为假设中的“表白”可以从前提中的“拥抱”中推断出来。

    前提：两个女人拥抱在一起。
    假设：两个女人在示爱。

下面是一个“矛盾”的例子，因为“运行编码示例”表示“不睡觉”，而不是“睡觉”。

    前提：一名男子正在运行Dive Into Deep Learning的编码示例。
    假设：该男子正在睡觉。

第三个例子显示了一种“中性”关系，因为“正在为我们表演”这一事实无法推断出“出名”或“不出名”。

    前提：音乐家们正在为我们表演。
    假设：音乐家很有名。
自然语言推断一致是理解自然语言的中心话题。它有着广泛的应用，从信息检索到开放领域的问答。为了研究这个问题，我们将首先研究一个流行的自然语言推断基准数据集。

# 二、斯坦福自然语言推断（SNLI）数据集

斯坦福自然语言推断语料库（Stanford Natural Language Inference，SNLI）是由500000多个带标签的英语句子对组成的集合 (Bowman et al., 2015)。我们在路径../data/snli_1.0中下载并存储提取的SNLI数据集。

In [2]:
import os
import re
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
#@save
d2l.DATA_HUB['SNLI'] = (
    'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    '9fcde07509c7e87ec61c640c1b2753d9041758e4')

data_dir = d2l.download_extract('SNLI')

上边这个报错的解决参考：https://blog.csdn.net/weixin_44562957/article/details/120116615

## 2.1 读取数据集

原始的SNLI数据集包含的信息比我们在实验中真正需要的信息丰富得多。因此，我们定义函数read_snli以仅提取数据集的一部分，然后返回前提、假设及其标签的列表。

In [9]:
#@save
def read_snli(data_dir, is_train):
    """将SNLI数据集解析为前提、假设和标签"""
    def extract_text(s):
        # 删除我们不会使用的信息
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # 用一个空格替换两个或多个连续的空格
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] \
                in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels

现在让我们打印前3对前提和假设，以及它们的标签（“0”“1”和“2”分别对应于“蕴涵”“矛盾”和“中性”）。

In [5]:
data_dir="..\data\snli_1.0"
train_data = read_snli(data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('前提：', x0)
    print('假设：', x1)
    print('标签：', y)

前提： A person on a horse jumps over a broken down airplane .
假设： A person is training his horse for a competition .
标签： 2
前提： A person on a horse jumps over a broken down airplane .
假设： A person is at a diner , ordering an omelette .
标签： 1
前提： A person on a horse jumps over a broken down airplane .
假设： A person is outdoors , on a horse .
标签： 0


训练集约有550000对，测试集约有10000对。下面显示了训练集和测试集中的三个标签“蕴涵”“矛盾”和“中性”是平衡的。

In [6]:
test_data = read_snli(data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

[183416, 183187, 182764]
[3368, 3237, 3219]


## 2.2 定义用于加载数据集的类

下面我们来定义一个用于加载SNLI数据集的类。类构造函数中的变量num_steps指定文本序列的长度，使得每个小批量序列将具有相同的形状。换句话说，在较长序列中的前num_steps个标记之后的标记被截断，而特殊标记"<pad>"将被附加到较短的序列后，指导它们的长度变为num_steps。通过实现__getitem__功能，我们可以任意访问带有索引idx的前提、假设和标签。

In [11]:
#@save
class SNLIDataset(torch.utils.data.Dataset):
    """用于加载SNLI数据集的自定义数据集"""
    def __init__(self,dataset,num_steps,vocab=None):
        self.num_steps=num_steps
        all_premise_tokens=d2l.tokenize(dataset[0])
        all_hypothesis_tokens=d2l.tokenize(dataset[1])
        if vocab==None:
            # 构建词表，把频率<min_freq的词元替换成reserved_tokens
            self.vocab=d2l.Vocab(all_premise_tokens+all_hypothesis_tokens,
                                 min_freq=5,reserved_tokens=['<pad>'])
        else:
            self.vocab=vocab
        self.premises=self._pad(all_premise_tokens) #前提
        self.hypothesis=self._pad(all_hypothesis_tokens)    #推断
        self.labels=torch.tensor(dataset[2])    #标签
        print('read '+str(len(self.premises))+' examples')

    def _pad(self,lines):
        return torch.tensor(
            [d2l.truncate_pad(self.vocab[line], self.num_steps,self.vocab['<pad>'])
             for line in lines])

    def __getitem__(self, idx):
        return (self.premises[idx],self.hypothesis[idx]),self.labels[idx]

    def __len__(self):
        return len(self.premises)

## 2.3 整合代码

现在，我们可以调用read_snli函数和SNLIDataset类来下载SNLI数据集，并返回训练集和测试集的DataLoader实例，以及训练集的词表。值得注意的是，我们必须使用熊训练集构造的词表作为测试集的词表。因此，在训练集中训练的模型将不知道来自测试集的任何新词元。

In [12]:
#@save
def load_data_snli(batch_size,num_steps=50):
    """下载SNLI数据集并返回数据迭代器和词表"""
    num_workers=d2l.get_dataloader_workers()
    # data_dir=d2l.download_extract('SNLI')
    train_data=read_snli(data_dir,True)
    test_data=read_snli(data_dir,False)
    train_set=SNLIDataset(train_data,num_steps)
    test_set=SNLIDataset(test_data,num_steps,train_set.vocab)
    train_iter=torch.utils.data.DataLoader(train_set,batch_size,
                                           shuffle=True,num_workers=num_workers)
    test_iter=torch.utils.data.DataLoader(test_set,batch_size,
                                          shuffle=False,num_workers=num_workers)
    return train_iter,test_iter,train_set.vocab

在这里，我们将批量大小设置为128时，将序列长度设置为50，并调用load_data_snli函数来获取数据迭代器和词表。然后我们打印词表大小。

In [9]:
train_iter,test_iter,vocab=load_data_snli(128,50)
len(vocab)

read 549367 examples
read 9824 examples


18678

In [12]:
print(train_iter)

现在我们打印第一个批量的形状。与情感分析相反，我们有分别代表前提和假设的两个输入X[0]和X[1]。

In [ ]:
for X,Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break

## 补充

mini-batch 喂入模型数据的时候要保证mini-batch中句子长度一致，但是对于较短的句子，就需要使用特定的字符进行填充到统一的句子长度。但是我们不希望其填充的pad数据（一般为0）进入GRU或是LSTM模块，一是浪费资源，二是可能造成句子表征不准确。所以pack_padded_sequence 类应运而生。主要是对填充过的数据进行压缩。

# 三、自然语言推断：使用注意力

我们在 15.4节中介绍了自然语言推断任务和SNLI数据集。鉴于许多模型都是基于复杂而深度的架构，Parikh等人提出用注意力机制解决自然语言推断问题，并称之为“可分解注意力模型” (Parikh et al., 2016)。这使得模型没有循环层或卷积层，在SNLI数据集上以更少的参数实现了当时的最佳结果。本节将描述并实现这种基于注意力的自然语言推断方法（使用MLP），如 图15.5.1中所述。

![](https://files.mdnice.com/user/23981/9471edea-396c-4379-8d20-89dcbf3d79d9.png)

## 3.1 模型

与保留前提和假设中词元的顺序相比，我们可以将一个文本序列中的词元与另一个文本序列中的每个词元对齐，然后比较和聚合这些信息，以预测前提和假设之间的逻辑关系。与机器翻译中原句和目标句之间的词元对齐类似，前提和假设之间的词元对齐可以通过注意力机制灵活地完成。

>在自然语言处理任务中，对齐（Alignment）指的是将一个文本序列中的词元与另一个文本序列中的相应词元进行匹配。对齐的目的是建立两个文本序列之间的词元级别的对应关系，以便比较和聚合两个序列之间的信息。
具体实现对齐的方法取决于任务和数据。以下是几种常见的对齐方法：
>1. 基于规则的对齐：根据预定义的规则和启发式方法，将两个文本序列中的词元进行匹配。规则可以基于词元的相似性、词性标签、语法结构等进行定义。
>2. 基于词向量的对齐：使用词向量模型（如Word2Vec、GloVe、BERT等）将文本序列中的词元表示为向量，在向量空间中计算词元之间的相似度，并将相似度高的词元进行匹配。
>3. 基于注意力机制的对齐：使用注意力机制，将一个文本序列中的每个词元与另一个序列中的所有词元计算注意力权重，然后根据注意力权重对词元进行加权平均，从而获得两个序列的对齐表示。
>4. 基于机器翻译的对齐：将文本序列对齐问题视为机器翻译任务，其中一个序列作为源语言，另一个序列作为目标语言。使用机器翻译模型，将源语言序列翻译为目标语言序列，然后将翻译结果与目标语言序列进行对齐。
>这些方法都旨在将两个文本序列中的词元进行对应，从而捕捉两个序列之间的关系和信息。具体选择哪种对齐方法取决于任务的特点、数据的性质以及模型的需求。

![](https://files.mdnice.com/user/23981/3868d07e-c78b-4ef1-83fa-069a6811012f.png)

图15.5.2描述了使用注意力机制的自然语言推断方法。从高层次上讲，它由三个联合训练的步骤组成：对齐、比较和汇总。我们将在下面一步一步地对它们进行说明。

In [13]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## 3.2 注意

第一步是将一个文本序列中的词元与另一个序列中的每个词元对齐。假设前提是“我确实需要睡眠”，假设是“我累了”。由于语义上的相似性，我们不妨将假设中的“我”与前提中的“我”对齐，将假设中的“累”与前提中的“睡眠”对齐。同样，我们可能希望将前提中的“我”与假设中的“我”对齐，将前提中的“需要”和“睡眠”与假设中的“累”对齐。请注意，这种对齐是使用加权平均的“软”对齐，其中理想情况下较大的权重与要对齐的词元相关联。为了便于演示，图15.5.2以“硬”对齐的方式显示了这种对齐方式。

>在文本对齐中，有两种常见的对齐方式：硬对齐（Hard Alignment）和软对齐（Soft Alignment）。
>1. 硬对齐：硬对齐是一种严格的一对一对应关系，其中一个文本序列中的每个词元都与另一个文本序列中的唯一词元对齐。这意味着每个词元都有一个确定的对齐关系，不存在模糊或不确定的情况。在硬对齐中，对齐通常是通过规则或启发式方法进行确定，例如基于词性标签或语法结构的规则。
>2. 软对齐：软对齐是一种模糊的对应关系，其中一个文本序列中的词元可以与另一个文本序列中的多个词元进行对齐。这允许词元之间存在一定的重叠或模糊的对应关系。软对齐通常使用加权平均的方式进行，其中每个词元与对应的词元关联的权重表示对齐的强度或相似度。

现在，我们更详细的描述使用注意力机制的软对齐。用$\mathbf{A}=(\mathbf{a}_1,...,\mathbf{a}_m)$和$\mathbf{B}=(\mathbf{b}_1,...,\mathbf{b}_n)$表示前提和假设，其词元数量分别为m和n，其中$\mathbf{a}_i,\mathbf{b}_j\in\mathbb{R}^d(i=1,\ldots,m,j=1,\ldots,n)$是d维的词向量。对于软对齐，我们将注意力权重$e_{ij}\in\mathbb{R}$计算为：
$$e_{ij}=f(\mathbf{a}_i)^\top f(\mathbf{b}_j)$$
其中函数f是在下面的mlp函数中定义的多层感知机。输出维度f由mlp的num_hiddens参数指定。

In [13]:
def mlp(num_inputs,num_hiddens,flatten):
    """多层感知机"""
    net=[]
    net.append(nn.Dropout(0.2))
    net.append(nn.Linear(num_inputs,num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        net.append(nn.Flatten(start_dim=1))
    net.append(nn.Dropout(0.2))
    net.append(nn.Linear(num_hiddens,num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        net.append(nn.Flatten(start_dim=1))
    return nn.Sequential(*net)

值得注意的是，在（15.5.1）中，f分别输入$a_i$和$b_i$，而不是将它们一对放在一起作为输入。这种分解技巧导致f只有m + n个次计算（线性复杂度），而不是mn次计算（二次复杂度）。

对 (15.5.1)中的注意力权重进行规范化，我们计算假设中所有词元向量的加权平均值，以获得假设的表示，该假设与前提中索引i的词元进行软对齐：

$$\beta_i=\sum_{j=1}^n\frac{\exp(e_{ij})}{\sum_{k=1}^n\exp(e_{ik})}\mathbf{b}_j$$

同样，我们计算假设中索引为的每个词元与前提词元的软对齐：

$$\boldsymbol{\alpha}_j=\sum_{i=1}^m\frac{\exp(e_{ij})}{\sum_{k=1}^m\exp(e_{kj})}\mathbf{a}_i$$

下面，我们定义Attend类来计算假设（beta）与输入前提A的软对齐以及前提（alpha）与输入假设B的软对齐。

In [14]:
class Attend(nn.Module):
    def __init__(self, num_inputs, num_hiddens, **kwargs):
        super(Attend, self).__init__(**kwargs)
        self.f = mlp(num_inputs, num_hiddens, flatten=False)

    def forward(self, A, B):
        # A/B的形状：（批量大小，序列A/B的词元数，embed_size）
        # f_A/f_B的形状：（批量大小，序列A/B的词元数，num_hiddens）
        f_A = self.f(A)
        f_B = self.f(B)
        # e的形状：（批量大小，序列A的词元数，序列B的词元数）
        e = torch.bmm(f_A, f_B.permute(0, 2, 1))
        # beta的形状：（批量大小，序列A的词元数，embed_size），
        # 意味着序列B被软对齐到序列A的每个词元(beta的第1个维度)
        beta = torch.bmm(F.softmax(e, dim=-1), B)
        # beta的形状：（批量大小，序列B的词元数，embed_size），
        # 意味着序列A被软对齐到序列B的每个词元(alpha的第1个维度)
        alpha = torch.bmm(F.softmax(e.permute(0, 2, 1), dim=-1), A)
        return beta, alpha

## 3.3 比较

在下一步中，我们将一个序列中的词元与与该词元软对齐的另一个序列进行比较。请注意，在软对齐中，一个序列中的所有词元（尽管可能具有不同的注意力权重）将与另一个序列中的词元进行比较。为便于演示， 图15.5.2对词元以硬的方式对齐。例如，上述的注意（attending）步骤确定前提中的“need”和“sleep”都与假设中的“tired”对齐，则将对“疲倦-需要睡眠”进行比较。

在比较步骤中，我们将来自一个序列的词元的连结（运算符[ , ]）和来自另一序列的对齐的词元送入函数g（一个多层感知机）：

$$\begin{gathered}
\mathbf{v}_{A,i}=g([\mathbf{a}_{i},\boldsymbol{\beta}_{i}]),i =1,\ldots,m \\
\mathbf v_{B,j}=g([\mathbf b_{j},\boldsymbol\alpha_{j}]),j =1,\ldots,n
\end{gathered}$$

上式中，$v_{A,i}$是指，所有假设中的词元与前提中词元i软对齐，再与词元i的比较；而$v_{B,j}$是指，所有前提中的词元与假设中词元j软对齐，再与词元j的比较。下面的Compare类定义了比较步骤。

In [15]:
class Compare(nn.Module):
    def __init__(self,num_inputs,num_hiddens,**kwargs):
        super(Compare, self).__init__(**kwargs)
        self.g=mlp(num_inputs,num_hiddens,flatten=False)

    def forward(self,A,B,beta,alpha):
        V_A=self.g(torch.cat([A,beta],dim=2))
        V_B=self.g(torch.cat([B,alpha],dim=2))
        return V_A, V_B

## 3.4 聚合

现在我们有两组比较向量$\mathbf{v}_{A,i}(i=1,\ldots,m)$和$\mathbf{v}_{B,j}(j=1,\ldots,n)$。在最后异步中，我们将聚合这些信息以推断逻辑关系。恶魔首先求和这两组比较向量：

$$\mathbf{v}_{A}=\sum_{i=1}^{m}\mathbf{v}_{A,i}, \mathbf{v}_{B}=\sum_{j=1}^{n}\mathbf{v}_{B,j}$$

接下来，我们将两个求和结果的连结提供给函数h（一个多层感知机），以获得逻辑关系的分类结果：

$$\hat{\mathbf{y}}=h([\mathbf{v}_A,\mathbf{v}_B])$$

聚合步骤在以下Aggregate类中定义。

In [16]:
class Aggregate(nn.Module):
    def __init__(self,num_inputs,num_hiddens,num_outputs,**kwargs):
        super(Aggregate, self).__init__(**kwargs)
        self.h=mlp(num_inputs,num_hiddens,flatten=True)
        self.linear=nn.Linear(num_hiddens,num_outputs)

    def forward(self,V_A,V_B):
        # 对两组比较向量分别求和
        V_A=V_A.sum(dim=1)
        V_B=V_B.sum(dim=1)
        # 将两个求和结果连结送入多层感知机中
        Y_hat=self.linear(self.h(torch.cat([V_A,V_B],dim=1)))
        return Y_hat

## 3.5 整合代码

通过将注意力步骤、比较步骤和聚合步骤组合在一起，我们定义了可分解的注意力模型来联合训练这三个步骤。

可分解的注意力模型（Decomposable Attention Model）是一种注意力机制的变体，常用于文本匹配和语义关系建模任务。它与传统的注意力模型在结构上有所不同。

传统的注意力模型（例如，Bahdanau Attention和Luong Attention）通常用于机器翻译和语音识别等序列到序列的任务。这些模型使用了一个全局的注意力权重，它通过计算目标序列中的每个位置与源序列中所有位置的关联度来决定在目标位置上的注意力权重。这样的全局关注机制在序列对齐和信息聚合方面表现良好。

可分解的注意力模型则更适用于文本匹配和语义关系建模任务，其中需要将两个文本序列之间的关系进行建模。该模型的基本思想是将注意力机制分解为多个子注意力，分别在不同的层次上进行计算和建模。

具体来说，可分解的注意力模型由以下几个组成部分组成：

1. Attend函数：用于计算每个词元对于另一个文本序列中的所有词元的关注程度。这个函数可以是简单的点积操作、双线性操作或多层感知机等。

2. Compare函数：用于比较两个词元之间的关系和相似度，并生成一个相关性分数。这个函数通常是一个前馈神经网络。

3. Aggregate函数：用于将多个词元的相关性分数进行聚合，生成全局的注意力权重或匹配分数。这个函数可以是简单的求和、加权求和或者更复杂的聚合方式。

可分解的注意力模型通过分解注意力计算过程为多个子操作，可以更灵活地建模文本之间的关系。它能够将注意力机制应用于每个词元对之间的关系，而不仅仅局限于全局的对齐关系。这种分解的方式允许模型更好地捕捉局部的语义和句子级别的关联性，从而提升了建模文本关系的能力。

总结起来，可分解的注意力模型与传统的注意力模型在结构和应用场景上有所不同。可分解的注意力模型更适用于文本匹配和语义关系建模任务，通过分解注意力计算过程，更灵活地建模文本之间的关系。

In [17]:
class DecomposableAttention(nn.Module):
    def __init__(self,vocab,embed_size,num_hiddens,num_inputs_attend=100,
                 num_inputs_compare=200,num_inputs_agg=400,**kwargs):
        super(DecomposableAttention, self).__init__(**kwargs)
        self.embedding=nn.Embedding(len(vocab),embed_size)
        self.attend=Attend(num_inputs_attend,num_hiddens)
        self.compare=Compare(num_inputs_compare,num_hiddens)
        # 有3种可能的输出：蕴涵、矛盾和中性
        self.aggregate=Aggregate(num_inputs_agg,num_hiddens,num_outputs=3)

    def forward(self,X):
        premises,hypothesis=X
        A=self.embedding(premises)
        B=self.embedding(hypothesis)
        beta,alpha=self.attend(A,B)
        V_A,V_B=self.compare(A,B,beta,alpha)
        Y_hat=self.aggregate(V_A,V_B)
        return Y_hat

# 四、训练和评估模型

现在，我们将在SNLI数据集上对定义好的可分解注意力模型进行训练和评估。我们从读取数据集开始。

## 4.1 读取数据集

我们使用 15.4节中定义的函数下载并读取SNLI数据集。批量大小和序列长度分别设置为256和50。

In [18]:
data_dir="..\data\snli_1.0"
batch_size, num_steps = 256, 50
train_iter, test_iter, vocab = load_data_snli(batch_size, num_steps)

read 549367 examples
read 9824 examples


## 4.2 创建模型

我们使用预训练好的100维GloVe嵌入来表示输入词元。我们将向量$a_i$和$b_j$在（15.5.1）中的维数预定义为100。（15.5.1）中的函数f和（15.5.4）中的函数g的输出维度被设置为200.然后我们创建一个模型实例，初始化它的参数，并加载GloVe嵌入来初始化输入词元的向量。

In [20]:
embed_size, num_hiddens, devices = 100, 200, d2l.try_all_gpus()
net = DecomposableAttention(vocab, embed_size, num_hiddens)
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')
embeds = glove_embedding[vocab.idx_to_token]
net.embedding.weight.data.copy_(embeds)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.2871, -0.5999,  0.4688,  ...,  0.3639, -0.8138, -0.2211],
        [ 0.9682,  0.0718,  0.1710,  ...,  0.1024,  1.3345, -0.4854],
        [ 0.1964,  0.0332,  0.1263,  ...,  0.1077,  0.0260,  0.8928]])

## 4.3 训练和评估模型

与 12.5节中接受单一输入（如文本序列或图像）的split_batch函数不同，我们定义了一个split_batch_multi_inputs函数以小批量接受多个输入，如前提和假设。

现在我们可以在SNLI数据集上训练和评估模型。

In [ ]:
lr, num_epochs = 0.001, 4
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

## 4.4 使用模型

最后，定义预测函数，输出一对前提和假设之间的逻辑关系。

In [ ]:
#@save
def predict_snli(net,vocab,premise,hypothesis):
    """预测前提和假设之间的逻辑关系"""
    net.eval()
    premise=torch.tensor(vocab[premise],device=d2l.try_gpu())
    hypothesis=torch.tensor(vocab[hypothesis],device=d2l.try_gpu())
    label=torch.argmax(net([premise.reshape((1,-1)),
                            hypothesis.reshape((1,-1))]),dim=1)
    return 'entailment' if label==0 else 'contradiction' if label==1 else 'neutral'

In [ ]:
predict_snli(net, vocab, ['he', 'is', 'good', '.'], ['he', 'is', 'bad', '.'])